In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
from glob import glob
import os
import json 
import timm
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A

# PlatVillage

In [2]:
village_data = os.listdir('./data/public/PlantVillage')
label_encoder = {}
for idx, data_name in enumerate(village_data) :
    label_encoder[idx] = data_name

label_decoder = {val:key for key, val in label_encoder.items()}
display(label_decoder)
display(label_encoder)

{'Apple___Apple_scab': 0,
 'Apple___Black_rot': 1,
 'Apple___Cedar_apple_rust': 2,
 'Apple___healthy': 3,
 'Blueberry___healthy': 4,
 'Cherry_(including_sour)___healthy': 5,
 'Cherry_(including_sour)___Powdery_mildew': 6,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7,
 'Corn_(maize)___Common_rust_': 8,
 'Corn_(maize)___healthy': 9,
 'Corn_(maize)___Northern_Leaf_Blight': 10,
 'Grape___Black_rot': 11,
 'Grape___Esca_(Black_Measles)': 12,
 'Grape___healthy': 13,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 14,
 'Orange___Haunglongbing_(Citrus_greening)': 15,
 'Peach___Bacterial_spot': 16,
 'Peach___healthy': 17,
 'Pepper,_bell___Bacterial_spot': 18,
 'Pepper,_bell___healthy': 19,
 'Potato___Early_blight': 20,
 'Potato___healthy': 21,
 'Potato___Late_blight': 22,
 'Raspberry___healthy': 23,
 'Soybean___healthy': 24,
 'Squash___Powdery_mildew': 25,
 'Strawberry___healthy': 26,
 'Strawberry___Leaf_scorch': 27,
 'Tomato___Bacterial_spot': 28,
 'Tomato___Early_blight': 29,
 'Toma

{0: 'Apple___Apple_scab',
 1: 'Apple___Black_rot',
 2: 'Apple___Cedar_apple_rust',
 3: 'Apple___healthy',
 4: 'Blueberry___healthy',
 5: 'Cherry_(including_sour)___healthy',
 6: 'Cherry_(including_sour)___Powdery_mildew',
 7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 8: 'Corn_(maize)___Common_rust_',
 9: 'Corn_(maize)___healthy',
 10: 'Corn_(maize)___Northern_Leaf_Blight',
 11: 'Grape___Black_rot',
 12: 'Grape___Esca_(Black_Measles)',
 13: 'Grape___healthy',
 14: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 15: 'Orange___Haunglongbing_(Citrus_greening)',
 16: 'Peach___Bacterial_spot',
 17: 'Peach___healthy',
 18: 'Pepper,_bell___Bacterial_spot',
 19: 'Pepper,_bell___healthy',
 20: 'Potato___Early_blight',
 21: 'Potato___healthy',
 22: 'Potato___Late_blight',
 23: 'Raspberry___healthy',
 24: 'Soybean___healthy',
 25: 'Squash___Powdery_mildew',
 26: 'Strawberry___healthy',
 27: 'Strawberry___Leaf_scorch',
 28: 'Tomato___Bacterial_spot',
 29: 'Tomato___Early_blight',
 30: '

# custom dataset 정의

In [3]:
class VillageDataset(Dataset) :
    def __init__(self, files, transform, mode='train') :
        super(VillageDataset, self).__init__()
        self.files = files
        self.transform = transform
        self.mode = mode
        
    def __len__(self) :
        return len(self.files)
    
    def __getitem__(self, idx) :
        file_path = self.files[idx]
        
        label = label_decoder[file_path.split('\\')[-2]]
        
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)['image']
        img = img.transpose(2, 0, 1)
        
        return torch.tensor(img, dtype=torch.float32) / 255.0, torch.tensor(label, dtype=torch.long)

# Dataloader w/o kfold

In [4]:
train_transforms = A.Compose([
                A.Resize(224 ,224),
                A.OneOf([
                    A.Rotate(),
                    A.HorizontalFlip(),
                    A.VerticalFlip()
                ], p=1)
            ])

val_transforms = A.Compose([
    A.Resize(224,224)
])

train = glob('./data/public/PlantVillage/*/*.JPG')
print("total : ", len(train))
label_list = [label_decoder[img_path.split('\\')[-2]] for img_path in train]

train, val = train_test_split(train, test_size=0.2, shuffle=True, stratify=label_list)
print("train : ", len(train))
print("val : ", len(val))
display(train)
train_dataset = VillageDataset(train, train_transforms)
val_dataset = VillageDataset(val, val_transforms)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

total :  54304
train :  43443
val :  10861


['./data/public/PlantVillage\\Blueberry___healthy\\9039b699-aa44-4671-b63e-b9eba802976a___RS_HL 2505.JPG',
 './data/public/PlantVillage\\Soybean___healthy\\69fd19b2-93b5-4150-8ba9-0d624a6b00e4___RS_HL 3676.JPG',
 './data/public/PlantVillage\\Potato___Late_blight\\9570095c-02e9-497b-bec1-4e526f3462e5___RS_LB 2815.JPG',
 './data/public/PlantVillage\\Peach___Bacterial_spot\\767e3ee3-996b-48f9-b587-0b19e0eb3569___Rut._Bact.S 1451.JPG',
 './data/public/PlantVillage\\Tomato___Spider_mites Two-spotted_spider_mite\\0956a6ef-18ce-4a82-a9a1-ca41d07ae13f___Com.G_SpM_FL 9371.JPG',
 './data/public/PlantVillage\\Soybean___healthy\\2617acc7-e60c-4bed-abcc-64fdddf0d94c___RS_HL 6454.JPG',
 './data/public/PlantVillage\\Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot\\b53702bb-dfd5-4791-a550-d691cdb8a822___RS_GLSp 4379.JPG',
 './data/public/PlantVillage\\Cherry_(including_sour)___healthy\\472fd482-aec7-4e73-86d1-7161f6a5024b___JR_HL 4178.JPG',
 './data/public/PlantVillage\\Cherry_(including_sour)___Po

# DataLoader w/ kfold

In [5]:
img_list = glob('./data/public/PlantVillage/*/*.JPG')
label_list = [label_decoder[img_path.split('\\')[-2]] for img_path in img_list]

kfold = StratifiedKFold(n_splits=4, random_state=13, shuffle=True)
for idx, (kfold_train, kfold_val) in enumerate(kfold.split(img_list, label_list), 1) :
    print(idx, len(kfold_train), len(kfold_val))
    print(kfold_train)
    print(kfold_val)

1 15477 5159
[    1     2     3 ... 20632 20634 20635]
[    0    16    25 ... 20616 20630 20633]
2 15477 5159
[    0     1     3 ... 20633 20634 20635]
[    2     5    10 ... 20615 20617 20625]
3 15477 5159
[    0     1     2 ... 20633 20634 20635]
[    6    12    14 ... 20610 20622 20632]
4 15477 5159
[    0     2     5 ... 20630 20632 20633]
[    1     3     4 ... 20631 20634 20635]


# Plant Doc

### image 경로가 너무 길어서 읽어오지 못함.
- 이름을 0 ~ 1로 바꿈

In [43]:
img_name = glob('./data/public/PlantDoc/train/*.jpg')
len(img_name)

2328

In [44]:
with open('./data/public/PlantDoc/train/_classes.txt', 'r') as f :
    classese = f.readlines()

before_name = []
modi_name = []
for i in range(1, len(classese)) :
    split_name = classese[i].split(', ')[0]
    before_name.append(split_name)
    modi_name.append(f'{i}.jpg')
    classese[i] = classese[i].replace(split_name, f'{i}.jpg')
    

display(classese)
display(before_name)
display(modi_name)

['filename, Apple Scab Leaf, Apple leaf, Apple rust leaf, Bell_pepper leaf, Bell_pepper leaf spot, Blueberry leaf, Cherry leaf, Corn Gray leaf spot, Corn leaf blight, Corn rust leaf, Peach leaf, Potato leaf, Potato leaf early blight, Potato leaf late blight, Raspberry leaf, Soyabean leaf, Soybean leaf, Squash Powdery mildew leaf, Strawberry leaf, Tomato Early blight leaf, Tomato Septoria leaf spot, Tomato leaf, Tomato leaf bacterial spot, Tomato leaf late blight, Tomato leaf mosaic virus, Tomato leaf yellow virus, Tomato mold leaf, Tomato two spotted spider mites leaf, grape leaf, grape leaf black rot\n',
 '1.jpg, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0\n',
 '2.jpg, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0\n',
 '3.jpg, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0\n',
 '4.jpg, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

['2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B1_jpg.rf.00584c9d14b6fda3086f9a482675d610.jpg',
 'Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b93a38d296.jpg',
 'd-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017e4558f1b4baeadc32a13beb178a6e.jpg',
 '2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2e382cf11216938e6f49b7d65ed.jpg',
 'blueberry-leaf-isolated-white-background-blueberry-leaf-isolated-99227523_jpg.rf.00a825930e9396a34d6c3fc5cf86686e.jpg',
 'Apple-Scab-image-02_jpg.rf.00cbc9a108dbdaadf4232b5392e3d3c8.jpg',
 'Faske%20Southern%20rust%20of%20corn_jpg.rf.016116db58617610bb259b2965f41589.jpg',
 '14456_img_jpg.rf.0052b88d849e614c6037a2834800b4f8.jpg',
 '24154194959_fd4b42edde_b_jpg.rf.0012f9978a300c5635840700fa784280.jpg',
 'Septoria_tomato_Margaret%20B_p8170755_0002_25004_jpg.rf.00ddfef2668a53da256f207f9b844782.jpg',
 'DCDl_p8UQAAf2hP_jpg.rf.0192cb81cdbb3365925d7d47527de47a.jpg',
 'IMG_6029-001_1500x1500%253E_JPG?1403119544_jpg.rf.01aae68643601f970bad558834c627ae.jpg',
 '4-dogwood_pm_jpg.

['1.jpg',
 '2.jpg',
 '3.jpg',
 '4.jpg',
 '5.jpg',
 '6.jpg',
 '7.jpg',
 '8.jpg',
 '9.jpg',
 '10.jpg',
 '11.jpg',
 '12.jpg',
 '13.jpg',
 '14.jpg',
 '15.jpg',
 '16.jpg',
 '17.jpg',
 '18.jpg',
 '19.jpg',
 '20.jpg',
 '21.jpg',
 '22.jpg',
 '23.jpg',
 '24.jpg',
 '25.jpg',
 '26.jpg',
 '27.jpg',
 '28.jpg',
 '29.jpg',
 '30.jpg',
 '31.jpg',
 '32.jpg',
 '33.jpg',
 '34.jpg',
 '35.jpg',
 '36.jpg',
 '37.jpg',
 '38.jpg',
 '39.jpg',
 '40.jpg',
 '41.jpg',
 '42.jpg',
 '43.jpg',
 '44.jpg',
 '45.jpg',
 '46.jpg',
 '47.jpg',
 '48.jpg',
 '49.jpg',
 '50.jpg',
 '51.jpg',
 '52.jpg',
 '53.jpg',
 '54.jpg',
 '55.jpg',
 '56.jpg',
 '57.jpg',
 '58.jpg',
 '59.jpg',
 '60.jpg',
 '61.jpg',
 '62.jpg',
 '63.jpg',
 '64.jpg',
 '65.jpg',
 '66.jpg',
 '67.jpg',
 '68.jpg',
 '69.jpg',
 '70.jpg',
 '71.jpg',
 '72.jpg',
 '73.jpg',
 '74.jpg',
 '75.jpg',
 '76.jpg',
 '77.jpg',
 '78.jpg',
 '79.jpg',
 '80.jpg',
 '81.jpg',
 '82.jpg',
 '83.jpg',
 '84.jpg',
 '85.jpg',
 '86.jpg',
 '87.jpg',
 '88.jpg',
 '89.jpg',
 '90.jpg',
 '91.jpg',
 '92.jpg

### image 이름에 '?', ',' 있는거 제거

In [45]:
# with open('./data/public/PlantDoc/train/_classes.txt', 'r') as f :
#     classese = f.readlines()

# before_name = []
# modi_name = []
# for i in range(1, len(classese)) :
    
#     split_name = classese[i].split(', ')
    
#     if '?' in split_name[0] :
#         classese[i] = classese[i].replace(split_name[0], split_name[0].replace('?','_'))
        
#     if ',' in classese[i].split(', ')[0] :
#         before_name.append(split_name[0])
#         modi_name.append(split_name[0].replace(',',''))
#         classese[i] = classese[i].replace(before_name[-1], modi_name[-1])
        
#         print("@@@@@@@@@@@@@@@@@@")
#         print(before_name[-1])
#         print(modi_name[-1])
        

### 새로운 label txt 파일 생성

In [46]:
with open('./data/public/PlantDoc/train/_modi_classes.txt', 'w') as f :
    for data in classese:
        f.write(data)

### image 이름도 변경해주기

In [47]:
root_path = './data/public/PlantDoc/train'
img_list = glob(root_path+'/*.JPG')

for i in range(0, len(before_name)) :
    before_index = img_list.index(root_path+'\\'+before_name[i])
    img_list[before_index] = root_path+'\\'+modi_name[i]
    print(root_path+'\\'+before_name[i])
    print(root_path+'\\'+modi_name[i])
    print()
    
    os.rename(root_path+'\\'+before_name[i].replace('?','_'), root_path+'\\'+modi_name[i])

./data/public/PlantDoc/train\2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B1_jpg.rf.00584c9d14b6fda3086f9a482675d610.jpg
./data/public/PlantDoc/train\1.jpg

./data/public/PlantDoc/train\Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b93a38d296.jpg
./data/public/PlantDoc/train\2.jpg

./data/public/PlantDoc/train\d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017e4558f1b4baeadc32a13beb178a6e.jpg
./data/public/PlantDoc/train\3.jpg

./data/public/PlantDoc/train\2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2e382cf11216938e6f49b7d65ed.jpg
./data/public/PlantDoc/train\4.jpg

./data/public/PlantDoc/train\blueberry-leaf-isolated-white-background-blueberry-leaf-isolated-99227523_jpg.rf.00a825930e9396a34d6c3fc5cf86686e.jpg
./data/public/PlantDoc/train\5.jpg

./data/public/PlantDoc/train\Apple-Scab-image-02_jpg.rf.00cbc9a108dbdaadf4232b5392e3d3c8.jpg
./data/public/PlantDoc/train\6.jpg

./data/public/PlantDoc/train\Faske%20Southern%20rust%20of%20corn_jpg.rf.016116db58617610bb259b2965f41589.jpg
./data/publ

ValueError: './data/public/PlantDoc/train\\IMG_6029-001_1500x1500%253E_JPG?1403119544_jpg.rf.01aae68643601f970bad558834c627ae.jpg' is not in list

In [38]:
root_path = './data/public/PlantDoc/train'
img_list = glob(root_path+'/*.JPG')

for i in range(0, len(before_name)) :
    before_index = img_list.index(root_path+'\\'+before_name[i])
    img_list[before_index] = root_path+'\\'+modi_name[i]
    print(root_path+'\\'+before_name[i])
    print(root_path+'\\'+modi_name[i])
    print()
    
    os.rename(root_path+'\\'+before_name[i], root_path+'\\'+modi_name[i])

./data/public/PlantDoc/train\2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B1_jpg.rf.00584c9d14b6fda3086f9a482675d610.jpg
./data/public/PlantDoc/train\1.jpg

./data/public/PlantDoc/train\Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b93a38d296.jpg
./data/public/PlantDoc/train\2.jpg

./data/public/PlantDoc/train\d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017e4558f1b4baeadc32a13beb178a6e.jpg
./data/public/PlantDoc/train\3.jpg

./data/public/PlantDoc/train\2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2e382cf11216938e6f49b7d65ed.jpg
./data/public/PlantDoc/train\4.jpg

./data/public/PlantDoc/train\blueberry-leaf-isolated-white-background-blueberry-leaf-isolated-99227523_jpg.rf.00a825930e9396a34d6c3fc5cf86686e.jpg
./data/public/PlantDoc/train\5.jpg

./data/public/PlantDoc/train\Apple-Scab-image-02_jpg.rf.00cbc9a108dbdaadf4232b5392e3d3c8.jpg
./data/public/PlantDoc/train\6.jpg

./data/public/PlantDoc/train\Faske%20Southern%20rust%20of%20corn_jpg.rf.016116db58617610bb259b2965f41589.jpg
./data/publ

ValueError: './data/public/PlantDoc/train\\IMG_6029-001_1500x1500%253E_JPG?1403119544_jpg.rf.01aae68643601f970bad558834c627ae.jpg' is not in list

# data list 만들기
- none, 12, 17, 28번 클래스 삭제

In [11]:
import pandas as pd

label_csv = pd.read_csv('./data/public/PlantDoc/train/_modi_classes.txt')
columns_name = list(label_csv.columns)
columns_name.remove('filename')
columns_name.insert(0, 'None')

label_encoder = {idx : val.lstrip() for idx, val in enumerate(columns_name)}
label_decoder = {val : key for key, val in label_encoder.items()}
display(label_encoder)
display(label_decoder)

{0: 'None',
 1: 'Apple Scab Leaf',
 2: 'Apple leaf',
 3: 'Apple rust leaf',
 4: 'Bell_pepper leaf',
 5: 'Bell_pepper leaf spot',
 6: 'Blueberry leaf',
 7: 'Cherry leaf',
 8: 'Corn Gray leaf spot',
 9: 'Corn leaf blight',
 10: 'Corn rust leaf',
 11: 'Peach leaf',
 12: 'Potato leaf',
 13: 'Potato leaf early blight',
 14: 'Potato leaf late blight',
 15: 'Raspberry leaf',
 16: 'Soyabean leaf',
 17: 'Soybean leaf',
 18: 'Squash Powdery mildew leaf',
 19: 'Strawberry leaf',
 20: 'Tomato Early blight leaf',
 21: 'Tomato Septoria leaf spot',
 22: 'Tomato leaf',
 23: 'Tomato leaf bacterial spot',
 24: 'Tomato leaf late blight',
 25: 'Tomato leaf mosaic virus',
 26: 'Tomato leaf yellow virus',
 27: 'Tomato mold leaf',
 28: 'Tomato two spotted spider mites leaf',
 29: 'grape leaf',
 30: 'grape leaf black rot'}

{'None': 0,
 'Apple Scab Leaf': 1,
 'Apple leaf': 2,
 'Apple rust leaf': 3,
 'Bell_pepper leaf': 4,
 'Bell_pepper leaf spot': 5,
 'Blueberry leaf': 6,
 'Cherry leaf': 7,
 'Corn Gray leaf spot': 8,
 'Corn leaf blight': 9,
 'Corn rust leaf': 10,
 'Peach leaf': 11,
 'Potato leaf': 12,
 'Potato leaf early blight': 13,
 'Potato leaf late blight': 14,
 'Raspberry leaf': 15,
 'Soyabean leaf': 16,
 'Soybean leaf': 17,
 'Squash Powdery mildew leaf': 18,
 'Strawberry leaf': 19,
 'Tomato Early blight leaf': 20,
 'Tomato Septoria leaf spot': 21,
 'Tomato leaf': 22,
 'Tomato leaf bacterial spot': 23,
 'Tomato leaf late blight': 24,
 'Tomato leaf mosaic virus': 25,
 'Tomato leaf yellow virus': 26,
 'Tomato mold leaf': 27,
 'Tomato two spotted spider mites leaf': 28,
 'grape leaf': 29,
 'grape leaf black rot': 30}

In [12]:
root_path = './data/public/PlantDoc/train'

with open('./data/public/PlantDoc/train/_modi_classes.txt', 'r') as f :
    classese = f.readlines()

label_list = []
img_list = []
for i in range(1, len(classese)) :
    img_list.append(root_path + '/' + classese[i].split(', ')[0])
    if '1' in classese[i].split(', ') :
        label_list.append(classese[i].split(', ').index('1'))
    elif '1\n' in classese[i].split(', ') :
        label_list.append(classese[i].split(', ').index('1\n'))
    else : 
        label_list.append(0)

print(img_list)
print(label_list)

['./data/public/PlantDoc/train/2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B1_jpg.rf.00584c9d14b6fda3086f9a482675d610.jpg', './data/public/PlantDoc/train/Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b93a38d296.jpg', './data/public/PlantDoc/train/d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017e4558f1b4baeadc32a13beb178a6e.jpg', './data/public/PlantDoc/train/2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2e382cf11216938e6f49b7d65ed.jpg', './data/public/PlantDoc/train/blueberry-leaf-isolated-white-background-blueberry-leaf-isolated-99227523_jpg.rf.00a825930e9396a34d6c3fc5cf86686e.jpg', './data/public/PlantDoc/train/Apple-Scab-image-02_jpg.rf.00cbc9a108dbdaadf4232b5392e3d3c8.jpg', './data/public/PlantDoc/train/Faske%20Southern%20rust%20of%20corn_jpg.rf.016116db58617610bb259b2965f41589.jpg', './data/public/PlantDoc/train/14456_img_jpg.rf.0052b88d849e614c6037a2834800b4f8.jpg', './data/public/PlantDoc/train/24154194959_fd4b42edde_b_jpg.rf.0012f9978a300c5635840700fa784280.jpg', './data/public/Pla

# class 별 갯수
- 10개 이하 class는 삭제 해줌
- 삭제해야할 label - 0, 12, 17, 28 

In [13]:
root_path = './data/public/PlantDoc/train'

with open('./data/public/PlantDoc/train/_modi_classes.txt', 'r') as f :
    classese = f.readlines()

label_list = []
img_list = []
for i in range(1, len(classese)) :
    img_list.append(root_path + '/' + classese[i].split(', ')[0])
    if '1' in classese[i].split(', ') :
        label_list.append(classese[i].split(', ').index('1'))
    elif '1\n' in classese[i].split(', ') :
        label_list.append(classese[i].split(', ').index('1\n'))
    else : 
        label_list.append(0)
label_cnt = {k : 0 for k, v in label_decoder.items()}

for i in label_list :
    label_cnt[label_encoder[i]] += 1

display(label_cnt)

{'None': 10,
 'Apple Scab Leaf': 83,
 'Apple leaf': 82,
 'Apple rust leaf': 78,
 'Bell_pepper leaf': 53,
 'Bell_pepper leaf spot': 62,
 'Blueberry leaf': 103,
 'Cherry leaf': 47,
 'Corn Gray leaf spot': 61,
 'Corn leaf blight': 178,
 'Corn rust leaf': 106,
 'Peach leaf': 101,
 'Potato leaf': 3,
 'Potato leaf early blight': 103,
 'Potato leaf late blight': 94,
 'Raspberry leaf': 112,
 'Soyabean leaf': 57,
 'Soybean leaf': 1,
 'Squash Powdery mildew leaf': 124,
 'Strawberry leaf': 87,
 'Tomato Early blight leaf': 79,
 'Tomato Septoria leaf spot': 137,
 'Tomato leaf': 56,
 'Tomato leaf bacterial spot': 101,
 'Tomato leaf late blight': 99,
 'Tomato leaf mosaic virus': 44,
 'Tomato leaf yellow virus': 68,
 'Tomato mold leaf': 86,
 'Tomato two spotted spider mites leaf': 0,
 'grape leaf': 57,
 'grape leaf black rot': 56}

### class 삭제

In [14]:
label_cnt_ = {label_decoder[k] : v for k, v in label_cnt.items()}
display(label_cnt_)
remove_list = []
for k, v in label_cnt_.items() :
    for i in range(0, len(label_list)) :
        if label_list[i] == k and v <= 10 :
            remove_list.append(i)
            print(label_list[i])
            print(img_list[i])
            
display(remove_list)
remove_list.sort(reverse=True)
display(remove_list)

print(len(img_list))
print(len(label_list))
for remove_num in remove_list :
    del img_list[remove_num]
    del label_list[remove_num]

print(len(img_list))
print(len(label_list))


### 0, 12, 17, 28 삭제해야할 label

{0: 10,
 1: 83,
 2: 82,
 3: 78,
 4: 53,
 5: 62,
 6: 103,
 7: 47,
 8: 61,
 9: 178,
 10: 106,
 11: 101,
 12: 3,
 13: 103,
 14: 94,
 15: 112,
 16: 57,
 17: 1,
 18: 124,
 19: 87,
 20: 79,
 21: 137,
 22: 56,
 23: 101,
 24: 99,
 25: 44,
 26: 68,
 27: 86,
 28: 0,
 29: 57,
 30: 56}

0
./data/public/PlantDoc/train/270412tglr-wild-strawberry-flowers-and-leaf-patch_jpg.rf.11e3d6fa8e0828ac716e6e40f92ebd6b.jpg
0
./data/public/PlantDoc/train/Hydrangea+%2527Claudie%2527%252C+Powdery+Mildew_JPG_jpg.rf.241a2ba81fae4dd8f4d42a639bf4c542.jpg
0
./data/public/PlantDoc/train/tomato_D4a-TobRingspotVirus-1000077_zoom_jpg.rf.2e3bf56fca79319caad79756a5a3489f.jpg
0
./data/public/PlantDoc/train/Downy%20mildew_JPG_jpg.rf.5cd0d81448f536b690c687fe43801ad8.jpg
0
./data/public/PlantDoc/train/raspberries_jpg.rf.6f265fc0cc6e20357544cda8c1e08794.jpg
0
./data/public/PlantDoc/train/powdery-mildew-erysiphe-plantani-on-young-sycamore-leaf-b372wn_jpg.rf.9f2fd157b98d7080ee62fda831e913e9.jpg
0
./data/public/PlantDoc/train/Black+Raspberry+Leaves+3_jpg.rf.a5118930e06e0d086213803cfdda04b3.jpg
0
./data/public/PlantDoc/train/raspberry-db_jpg.rf.b3eabfd7a40d314ed0bd68e867f3bf6b.jpg
0
./data/public/PlantDoc/train/aquilegia-powdery-mildew-erysiphe-aquilegiae-on-columbine-leaves-bga5xr_jpg.rf.bd049474ad6eb0e

[151, 324, 415, 827, 996, 1433, 1500, 1629, 1717, 1847, 1471, 1706, 1916, 36]

[1916, 1847, 1717, 1706, 1629, 1500, 1471, 1433, 996, 827, 415, 324, 151, 36]

2328
2328
2314
2314


### 삭제된 라벨만큼 라벨 번호 당기기

In [15]:
for i in range(0, len(label_list)) :
    print(label_list[i])
    if label_list[i] > 0 and label_list[i] < 12 :
        label_list[i] -= 1
    elif label_list[i] > 12 and label_list[i] < 17 :
        label_list[i] -= 2
    elif label_list[i] > 17 and label_list[i] < 28 :
        label_list[i] -= 3
    elif label_list[i] > 28 :
        label_list[i] -= 4
    print(label_list[i])
    print()

24
21

9
8

27
24

9
8

6
5

1
0

10
9

26
23

9
8

21
18

30
26

5
4

5
4

9
8

4
3

5
4

1
0

13
11

23
20

3
2

15
13

25
22

9
8

18
15

14
12

18
15

9
8

20
17

9
8

13
11

14
12

20
17

21
18

20
17

24
21

11
10

16
14

1
0

8
7

15
13

26
23

27
24

18
15

23
20

15
13

20
17

2
1

18
15

6
5

24
21

10
9

18
15

3
2

19
16

18
15

11
10

26
23

11
10

27
24

13
11

1
0

7
6

3
2

2
1

15
13

26
23

14
12

19
16

21
18

13
11

20
17

14
12

25
22

15
13

23
20

26
23

24
21

25
22

6
5

11
10

6
5

2
1

1
0

15
13

15
13

1
0

14
12

6
5

10
9

15
13

14
12

18
15

16
14

14
12

2
1

6
5

21
18

26
23

13
11

15
13

22
19

16
14

13
11

21
18

13
11

18
15

21
18

25
22

19
16

16
14

21
18

21
18

6
5

24
21

16
14

9
8

9
8

9
8

3
2

26
23

18
15

11
10

13
11

30
26

13
11

10
9

9
8

6
5

11
10

8
7

21
18

5
4

23
20

2
1

15
13

24
21

26
23

30
26

24
21

18
15

4
3

27
24

14
12

26
23

7
6

6
5

13
11

9
8

18
15

6
5

26
23

2
1

9
8

27
24

8
7

14
12

27
24

1
0




29
25

15
13

8
7

24
21

11
10

26
23

27
24

25
22

9
8

18
15

5
4

3
2

23
20

29
25

5
4

19
16

15
13

19
16

27
24

23
20

10
9

9
8

23
20

21
18

5
4

22
19

19
16

29
25

10
9

21
18

29
25

14
12

8
7

4
3

21
18

9
8

18
15

21
18

10
9

3
2

15
13

6
5

3
2

2
1

24
21

6
5

13
11

26
23

1
0

18
15

9
8

4
3

19
16

6
5

3
2

3
2

15
13

3
2

18
15

11
10

27
24

13
11

13
11

11
10

22
19

18
15

5
4

26
23

8
7

27
24

29
25

23
20

10
9

10
9

11
10

6
5

7
6

4
3

27
24

25
22

18
15

14
12

20
17

21
18

1
0

29
25

7
6

3
2

27
24

3
2

6
5

20
17

24
21

13
11

11
10

3
2

8
7

9
8

27
24

5
4

14
12

4
3

30
26

7
6

1
0

30
26

24
21

3
2

18
15

9
8

25
22

11
10

19
16

14
12

18
15

9
8

20
17

30
26

26
23

18
15

23
20

5
4

18
15

20
17

18
15

15
13

7
6

14
12

18
15

14
12

9
8

1
0

10
9

4
3

16
14

11
10

6
5

13
11

18
15

22
19

18
15

26
23

15
13

14
12

4
3

15
13

21
18

10
9

23
20

16
14

14
12

6
5

20
17

23
20

5
4

2
1

4
3

26
23

20
17


### label_decoder encoder 재선언

In [16]:
import pandas as pd

label_csv = pd.read_csv('./data/public/PlantDoc/train/_modi_classes.txt')
columns_name = list(label_csv.columns)
columns_name.remove('filename')
columns_name.remove(' Potato leaf') # 12
columns_name.remove(' Soybean leaf') # 17
columns_name.remove(' Tomato two spotted spider mites leaf') # 28

label_encoder = {idx : val.lstrip() for idx, val in enumerate(columns_name)}
label_decoder = {val : key for key, val in label_encoder.items()}
display(label_encoder)
display(label_decoder)

{0: 'Apple Scab Leaf',
 1: 'Apple leaf',
 2: 'Apple rust leaf',
 3: 'Bell_pepper leaf',
 4: 'Bell_pepper leaf spot',
 5: 'Blueberry leaf',
 6: 'Cherry leaf',
 7: 'Corn Gray leaf spot',
 8: 'Corn leaf blight',
 9: 'Corn rust leaf',
 10: 'Peach leaf',
 11: 'Potato leaf early blight',
 12: 'Potato leaf late blight',
 13: 'Raspberry leaf',
 14: 'Soyabean leaf',
 15: 'Squash Powdery mildew leaf',
 16: 'Strawberry leaf',
 17: 'Tomato Early blight leaf',
 18: 'Tomato Septoria leaf spot',
 19: 'Tomato leaf',
 20: 'Tomato leaf bacterial spot',
 21: 'Tomato leaf late blight',
 22: 'Tomato leaf mosaic virus',
 23: 'Tomato leaf yellow virus',
 24: 'Tomato mold leaf',
 25: 'grape leaf',
 26: 'grape leaf black rot'}

{'Apple Scab Leaf': 0,
 'Apple leaf': 1,
 'Apple rust leaf': 2,
 'Bell_pepper leaf': 3,
 'Bell_pepper leaf spot': 4,
 'Blueberry leaf': 5,
 'Cherry leaf': 6,
 'Corn Gray leaf spot': 7,
 'Corn leaf blight': 8,
 'Corn rust leaf': 9,
 'Peach leaf': 10,
 'Potato leaf early blight': 11,
 'Potato leaf late blight': 12,
 'Raspberry leaf': 13,
 'Soyabean leaf': 14,
 'Squash Powdery mildew leaf': 15,
 'Strawberry leaf': 16,
 'Tomato Early blight leaf': 17,
 'Tomato Septoria leaf spot': 18,
 'Tomato leaf': 19,
 'Tomato leaf bacterial spot': 20,
 'Tomato leaf late blight': 21,
 'Tomato leaf mosaic virus': 22,
 'Tomato leaf yellow virus': 23,
 'Tomato mold leaf': 24,
 'grape leaf': 25,
 'grape leaf black rot': 26}

### 라벨 카운트

In [5]:
label_cnt = {k : 0 for k, v in label_decoder.items()}

for i in label_list :
    label_cnt[label_encoder[i]] += 1

display(label_cnt)

{'Apple___Apple_scab': 630,
 'Apple___Black_rot': 621,
 'Apple___Cedar_apple_rust': 275,
 'Apple___healthy': 1645,
 'Blueberry___healthy': 1502,
 'Cherry_(including_sour)___healthy': 854,
 'Cherry_(including_sour)___Powdery_mildew': 1052,
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 513,
 'Corn_(maize)___Common_rust_': 1192,
 'Corn_(maize)___healthy': 1162,
 'Corn_(maize)___Northern_Leaf_Blight': 985,
 'Grape___Black_rot': 1180,
 'Grape___Esca_(Black_Measles)': 1383,
 'Grape___healthy': 423,
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)': 1076,
 'Orange___Haunglongbing_(Citrus_greening)': 5507,
 'Peach___Bacterial_spot': 2297,
 'Peach___healthy': 360,
 'Pepper,_bell___Bacterial_spot': 997,
 'Pepper,_bell___healthy': 1477,
 'Potato___Early_blight': 1000,
 'Potato___healthy': 152,
 'Potato___Late_blight': 1000,
 'Raspberry___healthy': 371,
 'Soybean___healthy': 5090,
 'Squash___Powdery_mildew': 1835,
 'Strawberry___healthy': 456,
 'Strawberry___Leaf_scorch': 1109,
 'Tomato___Bac

# Custom dataset 정의

In [18]:
class DocDataset(Dataset) :
    def __init__(self, imgs, labels, transform) :
        super(VillageDataset, self).__init__()
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        
    def __len__(self) :
        assert len(self.imgs) == len(self.labels)
        return len(self.imgs)
    
    def __getitem__(self, idx) :
        
        label = self.labels[idx]
        
        img = cv2.imread(self.imgs[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)['image']
        img = img.transpose(2, 0, 1)
        
        return torch.tensor(img, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# DataLoader w/o kfold

In [19]:
train_transforms = A.Compose([
                A.Resize(224 ,224),
                A.OneOf([
                    A.Rotate(),
                    A.HorizontalFlip(),
                    A.VerticalFlip()
                ], p=1)
            ])

val_transforms = A.Compose([
    A.Resize(224,224)
])


train_img, val_img, train_label, val_label = train_test_split(img_list, label_list, test_size=0.3, shuffle=True, stratify=label_list)

print("train_img : ", len(train_img))
print("train_lable : ", len(train_label))

print("val_img : ", len(val_img))
print("val_lable : ", len(val_label))

print(train_img[572])
print(train_label[572])

# train_dataset = VillageDataset(train, train_transforms)
# val_dataset = VillageDataset(val, val_transforms)

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

train_img :  1619
train_lable :  1619
val_img :  695
val_lable :  695
./data/public/PlantDoc/train/potato-early-blight-alternaria-alternata-target-spot-lesion-on-a-potato-leaf-x63fwh_jpg.rf.4bc8edaa7926474ba7a3d757f7aca91c.jpg
11


# Doc +Village = 라벨 합치기
- Doc == Village
- 11: 'Potato leaf early blight', ==  2: 'Potato___Early_blight',
- 12: 'Potato leaf late blight', ==  4: 'Potato___Late_blight',
- 17: 'Tomato Early blight leaf', ==  6: 'Tomato_Early_blight',
- 18: 'Tomato Septoria leaf spot', ==  10: 'Tomato_Septoria_leaf_spot',
- 19: 'Tomato leaf', ==  7: 'Tomato_healthy',
- 20: 'Tomato leaf bacterial spot', ==  5: 'Tomato_Bacterial_spot',
- 21: 'Tomato leaf late blight', ==  8: 'Tomato_Late_blight',
- 22: 'Tomato leaf mosaic virus', ==  13: 'Tomato__Tomato_mosaic_virus',
- 23: 'Tomato leaf yellow virus', ==  14: 'Tomato__Tomato_YellowLeaf__Curl_Virus'

## Village img, label data

In [20]:
village_data = os.listdir('./data/public/PlantVillage')
vill_label_encoder = {}
for idx, data_name in enumerate(village_data) :
    vill_label_encoder[idx] = data_name

vill_label_decoder = {val:key for key, val in vill_label_encoder.items()}
display(vill_label_decoder)
display(vill_label_encoder)

vill_img_list = glob('./data/public/PlantVillage/*/*.JPG')
vill_label_list = [vill_label_decoder[img_path.split('\\')[-2]] for img_path in vill_img_list]

print(len(vill_img_list))
print(len(vill_label_list))

{'Pepper__bell___Bacterial_spot': 0,
 'Pepper__bell___healthy': 1,
 'Potato___Early_blight': 2,
 'Potato___healthy': 3,
 'Potato___Late_blight': 4,
 'Tomato_Bacterial_spot': 5,
 'Tomato_Early_blight': 6,
 'Tomato_healthy': 7,
 'Tomato_Late_blight': 8,
 'Tomato_Leaf_Mold': 9,
 'Tomato_Septoria_leaf_spot': 10,
 'Tomato_Spider_mites_Two_spotted_spider_mite': 11,
 'Tomato__Target_Spot': 12,
 'Tomato__Tomato_mosaic_virus': 13,
 'Tomato__Tomato_YellowLeaf__Curl_Virus': 14}

{0: 'Pepper__bell___Bacterial_spot',
 1: 'Pepper__bell___healthy',
 2: 'Potato___Early_blight',
 3: 'Potato___healthy',
 4: 'Potato___Late_blight',
 5: 'Tomato_Bacterial_spot',
 6: 'Tomato_Early_blight',
 7: 'Tomato_healthy',
 8: 'Tomato_Late_blight',
 9: 'Tomato_Leaf_Mold',
 10: 'Tomato_Septoria_leaf_spot',
 11: 'Tomato_Spider_mites_Two_spotted_spider_mite',
 12: 'Tomato__Target_Spot',
 13: 'Tomato__Tomato_mosaic_virus',
 14: 'Tomato__Tomato_YellowLeaf__Curl_Virus'}

20636
20636


## Doc img, label data

In [21]:
import copy
doc_img_list = copy.deepcopy(img_list)
doc_label_list = copy.deepcopy(label_list)

doc_label_decoder = copy.deepcopy(label_decoder) 
doc_label_encoder = copy.deepcopy(label_encoder)

display(doc_label_decoder)
display(doc_label_encoder)
print(len(doc_img_list))
print(len(doc_label_list))

{'Apple Scab Leaf': 0,
 'Apple leaf': 1,
 'Apple rust leaf': 2,
 'Bell_pepper leaf': 3,
 'Bell_pepper leaf spot': 4,
 'Blueberry leaf': 5,
 'Cherry leaf': 6,
 'Corn Gray leaf spot': 7,
 'Corn leaf blight': 8,
 'Corn rust leaf': 9,
 'Peach leaf': 10,
 'Potato leaf early blight': 11,
 'Potato leaf late blight': 12,
 'Raspberry leaf': 13,
 'Soyabean leaf': 14,
 'Squash Powdery mildew leaf': 15,
 'Strawberry leaf': 16,
 'Tomato Early blight leaf': 17,
 'Tomato Septoria leaf spot': 18,
 'Tomato leaf': 19,
 'Tomato leaf bacterial spot': 20,
 'Tomato leaf late blight': 21,
 'Tomato leaf mosaic virus': 22,
 'Tomato leaf yellow virus': 23,
 'Tomato mold leaf': 24,
 'grape leaf': 25,
 'grape leaf black rot': 26}

{0: 'Apple Scab Leaf',
 1: 'Apple leaf',
 2: 'Apple rust leaf',
 3: 'Bell_pepper leaf',
 4: 'Bell_pepper leaf spot',
 5: 'Blueberry leaf',
 6: 'Cherry leaf',
 7: 'Corn Gray leaf spot',
 8: 'Corn leaf blight',
 9: 'Corn rust leaf',
 10: 'Peach leaf',
 11: 'Potato leaf early blight',
 12: 'Potato leaf late blight',
 13: 'Raspberry leaf',
 14: 'Soyabean leaf',
 15: 'Squash Powdery mildew leaf',
 16: 'Strawberry leaf',
 17: 'Tomato Early blight leaf',
 18: 'Tomato Septoria leaf spot',
 19: 'Tomato leaf',
 20: 'Tomato leaf bacterial spot',
 21: 'Tomato leaf late blight',
 22: 'Tomato leaf mosaic virus',
 23: 'Tomato leaf yellow virus',
 24: 'Tomato mold leaf',
 25: 'grape leaf',
 26: 'grape leaf black rot'}

2314
2314


## Village -> Doc

### Village Label 번호 변경

In [22]:
 # {vill : doc}
same_label_list_encoder = {0:27, 1:28, 2:11, 3:29, 4:12, 5:20, 6:17, 7:19, 8:21, 9:30, 10:18, 11:31, 12:32, 13:22, 14:23}
for i in range(0, len(vill_label_list)) :
    vill_label_list[i] = same_label_list_encoder[vill_label_list[i]]

## add

In [23]:
sum_img_list = doc_img_list + vill_img_list
sum_label_list = doc_label_list + vill_label_list

print(len(sum_img_list))
print(len(sum_label_list))

22950
22950


## sum_label_encoder decoder 선언

In [24]:
same_label_list_encoder = {0:27, 1:28, 2:11, 3:29, 4:12, 5:20, 6:17, 7:19, 8:21, 9:30, 10:18, 11:31, 12:32, 13:22, 14:23}
same_label_list_decoder = {v : k for k, v in same_label_list_encoder.items()}

additional_list = [0, 1, 3, 9, 11, 12]

sum_label_encoder = {}
for i in range(0, 33) :
    if i in same_label_list_decoder and i in doc_label_encoder :
        sum_label_encoder[i] = doc_label_encoder[i]
#         sum_label_encoder[vill_label_encoder[i]] = same_label_list_encoder[i]

    elif i in same_label_list_decoder and i not in doc_label_encoder :
        sum_label_encoder[i] = vill_label_encoder[same_label_list_decoder[i]]
#         sum_label_encoder[doc_label_encoder[i]] = same_label_list_encoder[i]
        
    elif i not in same_label_list_decoder and i in doc_label_encoder :
        sum_label_encoder[i] = doc_label_encoder[i]
        
sum_label_decoder = {v : k for k, v in sum_label_encoder.items()}

display(sum_label_encoder)
display(sum_label_decoder)

{0: 'Apple Scab Leaf',
 1: 'Apple leaf',
 2: 'Apple rust leaf',
 3: 'Bell_pepper leaf',
 4: 'Bell_pepper leaf spot',
 5: 'Blueberry leaf',
 6: 'Cherry leaf',
 7: 'Corn Gray leaf spot',
 8: 'Corn leaf blight',
 9: 'Corn rust leaf',
 10: 'Peach leaf',
 11: 'Potato leaf early blight',
 12: 'Potato leaf late blight',
 13: 'Raspberry leaf',
 14: 'Soyabean leaf',
 15: 'Squash Powdery mildew leaf',
 16: 'Strawberry leaf',
 17: 'Tomato Early blight leaf',
 18: 'Tomato Septoria leaf spot',
 19: 'Tomato leaf',
 20: 'Tomato leaf bacterial spot',
 21: 'Tomato leaf late blight',
 22: 'Tomato leaf mosaic virus',
 23: 'Tomato leaf yellow virus',
 24: 'Tomato mold leaf',
 25: 'grape leaf',
 26: 'grape leaf black rot',
 27: 'Pepper__bell___Bacterial_spot',
 28: 'Pepper__bell___healthy',
 29: 'Potato___healthy',
 30: 'Tomato_Leaf_Mold',
 31: 'Tomato_Spider_mites_Two_spotted_spider_mite',
 32: 'Tomato__Target_Spot'}

{'Apple Scab Leaf': 0,
 'Apple leaf': 1,
 'Apple rust leaf': 2,
 'Bell_pepper leaf': 3,
 'Bell_pepper leaf spot': 4,
 'Blueberry leaf': 5,
 'Cherry leaf': 6,
 'Corn Gray leaf spot': 7,
 'Corn leaf blight': 8,
 'Corn rust leaf': 9,
 'Peach leaf': 10,
 'Potato leaf early blight': 11,
 'Potato leaf late blight': 12,
 'Raspberry leaf': 13,
 'Soyabean leaf': 14,
 'Squash Powdery mildew leaf': 15,
 'Strawberry leaf': 16,
 'Tomato Early blight leaf': 17,
 'Tomato Septoria leaf spot': 18,
 'Tomato leaf': 19,
 'Tomato leaf bacterial spot': 20,
 'Tomato leaf late blight': 21,
 'Tomato leaf mosaic virus': 22,
 'Tomato leaf yellow virus': 23,
 'Tomato mold leaf': 24,
 'grape leaf': 25,
 'grape leaf black rot': 26,
 'Pepper__bell___Bacterial_spot': 27,
 'Pepper__bell___healthy': 28,
 'Potato___healthy': 29,
 'Tomato_Leaf_Mold': 30,
 'Tomato_Spider_mites_Two_spotted_spider_mite': 31,
 'Tomato__Target_Spot': 32}

# === Village Dataset으로만 훈련 ===

### Cutmix

In [25]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)
 
    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

### Model

In [26]:
import timm

class DeiT(nn.Module):
    def __init__(self, model_name, n_classes):
        super(DeiT, self).__init__()
        self.model = timm.create_model(model_name, num_classes=n_classes, pretrained=True)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

In [27]:
def accuracy_function(real, pred):    
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    score = f1_score(real, pred, average='macro')
    return score

def train_step(batch_item, training):
    img = batch_item[0].to(device)
    label = batch_item[1].to(device)
    print("img")
    print(img.shape)
    lam = np.random.beta(1.0, 1.0)
    
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            # add - cutmix
            rand_index = torch.randperm(img.size()[0])
            target_a = label
            target_b = label[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            
            output = model(img)
            loss = criterion(output, target_a) * lam + criterion(output, target_b) * (1. - lam)
            
#             output = model(img, csv_feature)
#             loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        score = accuracy_function(label, output)
        return loss, score
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
        score = accuracy_function(label, output)
        return loss, score

In [28]:
device = torch.device("cuda:0")#("cpu")
batch_size = 16
n_classes = 33
model_name = 'deit_small_patch16_224'
learning_rate = 1e-4
epochs = 20
save_path = 'public_vill_doc_pretrain_Deit_S16patch224.pt'

# Custom dataset 정의

In [29]:
class CombineDataset(Dataset) :
    def __init__(self, imgs, labels, transform) :
        super(CombineDataset, self).__init__()
        self.imgs = imgs
        self.labels = labels
        self.transform = transform
        
    def __len__(self) :
        assert len(self.imgs) == len(self.labels)
        return len(self.imgs)
    
    def __getitem__(self, idx) :
        
        label = self.labels[idx]
        
        img = cv2.imread(self.imgs[idx])
        print("???")
        print(img)
        print(self.imgs[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(image=img)['image']
        img = img.transpose(2, 0, 1)
        
        return torch.tensor(img, dtype=torch.float32)/255.0, torch.tensor(label, dtype=torch.long)

# DataLoader w/o kfold

In [30]:
train_transforms = A.Compose([
                A.Resize(224 ,224),
                A.OneOf([
                    A.Rotate(),
                    A.HorizontalFlip(),
                    A.VerticalFlip()
                ], p=1)
            ])

val_transforms = A.Compose([
    A.Resize(224,224)
])


print(len(sum_img_list))
print(len(sum_label_list))

train_img, val_img, train_label, val_label = train_test_split(sum_img_list, 
                                                              sum_label_list, 
                                                              test_size=0.2, 
                                                              shuffle=True, 
                                                              stratify=sum_label_list)

print("train_img : ", len(train_img))
print("train_lable : ", len(train_label))

print("val_img : ", len(val_img))
print("val_lable : ", len(val_label))

print(train_img[572])
print(train_label[572])

train_dataset = CombineDataset(train_img, train_label, train_transforms)
val_dataset = CombineDataset(val_img, val_label, val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

22950
22950
train_img :  18360
train_lable :  18360
val_img :  4590
val_lable :  4590
./data/public/PlantVillage\Tomato_Spider_mites_Two_spotted_spider_mite\77d5f7bf-a888-43f2-8403-82a01a10bf12___Com.G_SpM_FL 9579.JPG
31


In [32]:
model = DeiT(model_name, n_classes)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [33]:
loss_plot, val_loss_plot = [], []
metric_plot, val_metric_plot = [], []

for epoch in range(epochs):
    total_loss, total_val_loss = 0, 0
    total_acc, total_val_acc = 0, 0

    tqdm_dataset = tqdm(enumerate(train_loader))
    training = True
    for batch, batch_item in tqdm_dataset:
        batch_loss, batch_acc = train_step(batch_item, training)
        total_loss += batch_loss
        total_acc += batch_acc

        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(batch_loss.item()),
            'Mean Loss' : '{:06f}'.format(total_loss/(batch+1)),
            'Mean F-1' : '{:06f}'.format(total_acc/(batch+1))
        })
    loss_plot.append(total_loss/(batch+1))
    metric_plot.append(total_acc/(batch+1))

    tqdm_dataset = tqdm(enumerate(val_loader))
    training = False
    for batch, batch_item in tqdm_dataset:
        batch_loss, batch_acc = train_step(batch_item, training)
        total_val_loss += batch_loss
        total_val_acc += batch_acc

        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Val Loss': '{:06f}'.format(batch_loss.item()),
            'Mean Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
            'Mean Val F-1' : '{:06f}'.format(total_val_acc/(batch+1))
        })
    val_loss_plot.append(total_val_loss/(batch+1))
    val_metric_plot.append(total_val_acc/(batch+1))

    if np.max(val_metric_plot) == val_metric_plot[-1]:
        torch.save(model.state_dict(), f'{epoch}_{save_path}')

0it [00:00, ?it/s]

???
[[[138 132 151]
  [115 109 128]
  [136 130 149]
  ...
  [115 109 128]
  [134 128 147]
  [134 128 147]]

 [[150 144 163]
  [131 125 144]
  [147 141 160]
  ...
  [124 118 137]
  [160 154 173]
  [150 144 163]]

 [[132 126 145]
  [124 118 137]
  [145 139 158]
  ...
  [116 110 129]
  [103  97 116]
  [117 111 130]]

 ...

 [[162 152 168]
  [164 154 170]
  [167 157 173]
  ...
  [159 149 165]
  [148 138 154]
  [172 162 178]]

 [[154 144 160]
  [161 151 167]
  [156 146 162]
  ...
  [141 131 147]
  [142 132 148]
  [176 166 182]]

 [[170 160 176]
  [162 152 168]
  [150 140 156]
  ...
  [157 147 163]
  [148 138 154]
  [149 139 155]]]
./data/public/PlantVillage\Potato___healthy\af15ac00-67ff-4114-ace8-6dca48b94665___RS_HL 1865.JPG
???
[[[181 165 176]
  [171 155 166]
  [196 181 189]
  ...
  [189 179 191]
  [161 151 163]
  [158 148 160]]

 [[184 168 179]
  [183 167 178]
  [210 195 203]
  ...
  [157 147 159]
  [169 159 171]
  [153 143 155]]

 [[192 177 185]
  [201 186 194]
  [187 174 182]
  ...
  

1it [00:01,  1.39s/it, Epoch=1, Loss=3.763376, Mean Loss=3.763376, Mean F-1=0.000000]

???
[[[ 90  86  85]
  [ 91  87  86]
  [ 91  87  86]
  ...
  [128 127 131]
  [135 134 138]
  [131 130 134]]

 [[ 84  80  79]
  [ 83  79  78]
  [ 86  82  81]
  ...
  [131 130 134]
  [123 122 126]
  [123 122 126]]

 [[ 88  84  83]
  [ 85  81  80]
  [ 89  85  84]
  ...
  [139 138 142]
  [123 122 126]
  [129 128 132]]

 ...

 [[ 48  48  48]
  [ 51  51  51]
  [ 53  53  53]
  ...
  [113 116 121]
  [111 114 119]
  [110 113 118]]

 [[ 53  53  53]
  [ 56  56  56]
  [ 55  55  55]
  ...
  [116 119 124]
  [115 118 123]
  [115 118 123]]

 [[ 56  56  56]
  [ 58  58  58]
  [ 55  55  55]
  ...
  [121 124 129]
  [121 124 129]
  [122 125 130]]]
./data/public/PlantVillage\Tomato_Late_blight\65576adf-cef4-4f1a-8b7c-881e8729bdbf___GHLB2 Leaf 147.JPG
???
[[[ 76 146 139]
  [ 80 150 143]
  [ 83 151 144]
  ...
  [ 13  71  66]
  [ 15  73  68]
  [ 18  76  71]]

 [[ 75 145 138]
  [ 78 148 141]
  [ 83 151 144]
  ...
  [ 21  79  74]
  [ 16  74  69]
  [  9  67  62]]

 [[ 82 152 145]
  [ 81 151 144]
  [ 80 150 143]
  

3it [00:01,  2.16it/s, Epoch=1, Loss=3.393680, Mean Loss=3.551410, Mean F-1=0.015873]

...

50it [00:10,  5.48it/s, Epoch=1, Loss=1.950105, Mean Loss=2.450798, Mean F-1=0.193847]

???
[[[184 112 188]
  [185 116 191]
  [189 120 195]
  ...
  [146 116 165]
  [151 121 170]
  [163 133 182]]

 [[186 114 190]
  [188 119 194]
  [192 123 198]
  ...
  [146 117 163]
  [150 120 169]
  [160 131 177]]

 [[186 114 190]
  [189 120 195]
  [194 125 200]
  ...
  [149 118 163]
  [152 120 167]
  [158 127 172]]

 ...

 [[174 152 117]
  [172 153 116]
  [149 132  93]
  ...
  [196 161 228]
  [192 157 224]
  [186 151 218]]

 [[182 160 132]
  [174 153 122]
  [146 127  92]
  ...
  [199 164 231]
  [197 163 228]
  [191 157 222]]

 [[186 162 138]
  [175 152 126]
  [144 125  92]
  ...
  [202 167 234]
  [201 167 232]
  [197 163 228]]]
./data/public/PlantDoc/train/1b_jpg.rf.ab0c246abee404d74b074808f01955ac.jpg
???
[[[135 118 121]
  [137 120 123]
  [133 116 119]
  ...
  [100  82  89]
  [126 108 115]
  [141 123 130]]

 [[190 173 176]
  [233 216 219]
  [191 174 177]
  ...
  [159 141 148]
  [129 111 118]
  [121 103 110]]

 [[175 158 161]
  [160 143 146]
  [230 213 216]
  ...
  [152 134 141]
  [ 90  

50it [00:10,  4.79it/s, Epoch=1, Loss=1.950105, Mean Loss=2.450798, Mean F-1=0.193847]


???
[[[202 204 214]
  [202 204 214]
  [203 205 215]
  ...
  [166 168 179]
  [173 175 186]
  [168 170 181]]

 [[196 198 208]
  [192 194 204]
  [188 190 200]
  ...
  [175 177 188]
  [175 177 188]
  [164 166 177]]

 [[185 187 197]
  [181 183 193]
  [179 181 191]
  ...
  [172 174 185]
  [169 171 182]
  [155 157 168]]

 ...

 [[103 108 129]
  [ 94  99 120]
  [ 88  93 114]
  ...
  [103 103 121]
  [106 106 124]
  [115 115 133]]

 [[119 124 145]
  [112 117 138]
  [105 110 131]
  ...
  [ 98  98 116]
  [100 100 118]
  [106 106 124]]

 [[109 114 135]
  [106 111 132]
  [103 108 129]
  ...
  [101 101 119]
  [102 102 120]
  [105 105 123]]]
./data/public/PlantVillage\Tomato_Spider_mites_Two_spotted_spider_mite\fff8949b-3721-4f42-a489-8d39753fe7bb___Com.G_SpM_FL 8656.JPG
???
[[[148 147 157]
  [146 145 155]
  [165 164 174]
  ...
  [150 150 164]
  [142 142 156]
  [137 137 151]]

 [[157 156 166]
  [160 159 169]
  [160 159 169]
  ...
  [152 152 166]
  [154 154 168]
  [137 137 151]]

 [[170 169 179]
  [177

error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-uzca7qz1\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [50]:
img = cv2.imread('./data/public/PlantDoc/train/leaves-isolated-on-white-background-picture-id537925565_k=6&m=537925565&s=612x612&w=0&h=Ogzc4YHTI4i4q9aiTIG6Zq2sAMv6xTKiDlCmYANlzK8=_jpg.rf.dab246267da7f3a21649231a184bc0c5.jpg')
img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
a = 'IMG_6029-001_1500x1500%253E_JPG?1403119544_jpg.rf.01aae68643601f970bad558834c627ae.jpg'

if '?' in a :
    print("asdf")

In [55]:
import timm

class DeiT_(nn.Module):
    def __init__(self, model_name, n_classes):
        super(DeiT_, self).__init__()
        self.model = timm.create_model(model_name, num_classes=n_classes, pretrained=False)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

model = DeiT_(model_name, 15)
model.load_state_dict(torch.load('./model/public_vill_pretrain_Deit_S16patch224.pt', map_location=device))
model.to(device).eval()


DeiT_(
  (model): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1

In [61]:
data_ = torch.rand(1,3,224,224).to(device)

pred = model(data_)
pred.shape

torch.Size([1, 15])